In [1]:
with open('AT_pair.txt', 'r') as f:
    ppi_pairs = f.readlines() 


In [2]:
# !mkdir CV

In [3]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import numpy as np
for rep in range(3):
    n_splits = 10
    y_all_1 =  np.zeros((int(len(ppi_pairs)/2),1))
    y_all_2 =  np.ones( ( len(ppi_pairs) - int(len(ppi_pairs)/2)   ,1))
    y_all = np.concatenate((y_all_1, y_all_2), axis=0)
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True)
    skf_v = StratifiedKFold(n_splits=5, shuffle=True)

    for i, (train, test) in enumerate(skf.split(ppi_pairs, y_all)):
         
        train_pairs_file = 'CV/train'+str(rep)+'-'+str(i)
        test_pairs_file = 'CV/test'+str(rep)+'-'+str(i)
        valid_pairs_file = 'CV/valid'+str(rep)+'-'+str(i)
        valid_len = int(train.shape[0]*0.1)
        train_valid_idx = np.arange(train.shape[0])
        np.random.shuffle(train_valid_idx)
         
         
        with open(train_pairs_file,'w') as f:
            for ind in train_valid_idx[:train.shape[0]-valid_len]:
                f.writelines(ppi_pairs[ind])
         
        with open(test_pairs_file,'w') as f:
            for ind in test:
                f.writelines(ppi_pairs[ind])
        
        with open(valid_pairs_file,'w') as f:
            for ind in train_valid_idx[train.shape[0]-valid_len:]:
                f.writelines(ppi_pairs[ind])

In [4]:
with open('sequence.txt', 'r') as f:
    all_seqs = f.readlines()

In [5]:
all_seqs[0]

'Q94CL9\tMSETKPKRDSEYEGSNIKRMRLDDDDDVLRSPTRTLSSSSSSSLAYSVSDSGGFCSVALSEEEDDHLSSSISSGCSSSETNEIATRLPFSDLEAHEISETEISTLLTNNFRKQGISSSENLGETAEMDSATTEMRDQRKTEKKKKMEKSPTQAELDDFFSAAERYEQKRFTEKYNYDIVNDTPLEGRYQWVSLKP\n'

In [6]:
prot2seq = {}
for line in all_seqs:
    protID, seqs = line.rstrip().split('\t')
    prot2seq[protID] = seqs

In [7]:
prot2seq['Q94CL9']

'MSETKPKRDSEYEGSNIKRMRLDDDDDVLRSPTRTLSSSSSSSLAYSVSDSGGFCSVALSEEEDDHLSSSISSGCSSSETNEIATRLPFSDLEAHEISETEISTLLTNNFRKQGISSSENLGETAEMDSATTEMRDQRKTEKKKKMEKSPTQAELDDFFSAAERYEQKRFTEKYNYDIVNDTPLEGRYQWVSLKP'

In [8]:
from six.moves import cPickle as pickle #for performance

 
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di

In [9]:
save_dict(prot2seq, 'ATprot2seq.pkl')

In [10]:
with open('go_term.txt', 'r') as f:
    all_goterms = f.readlines()

In [11]:
all_goterms[0]

'O81127\tGO:0005515,GO:0016607,GO:0005681,GO:0008270,GO:0003676,GO:0000398,GO:0000166\n'

In [12]:
prot2go= {}
for line in all_goterms:
    splittext = line.rstrip().split('\t')
    if len(splittext) == 2:
        protID, gos = line.rstrip().split('\t')
        prot2go[protID] = gos
    else:
        print(line)
        protID = splittext[0] 
        prot2go[protID] = ''

B2BDI6	

Q3E6N8	

Q1H5B6	

B9DFM6	

A8MR24	

Q3ED96	

Q1EC57	

A8MR96	

Q2HII6	

C0SUY4	

Q548D1	

A8MQJ8	

Q68KJ4	

Q08A56	

B3H6L9	

A4FVN8	

Q94AW9	



In [13]:
prot2go['Q1H5B6'] =  'GO:0005737,GO:0008180'    
prot2go['B2BDI6'] =  'GO:0005634,GO:0005737,GO:0042742,GO:0050832,GO:0080027'  
prot2go['Q1EC57'] =  'GO:0005515,GO:0008180,GO:0000338'    
prot2go['A8MR96'] =  'GO:0007165'    

prot2go['C0SUY4'] =  'GO:0006355,GO:0009909,GO:0010074,GO:0010193,GO:0030154,GO:0042127,GO:1901342'    
prot2go['Q548D1'] =  'GO:0005737,GO:0000338,GO:0008180,GO:0010387'    

prot2go['Q68KJ4'] =  'GO:2000028'    
prot2go['A4FVN8'] =  'GO:0000398,GO:0005576,GO:0071013'    
prot2go['Q94AW9'] =  'GO:0000178,GO:0003723,GO:0000176,GO:0000177,GO:0000467,GO:0034427,GO:0034475,GO:0043928,GO:0071034,GO:0071035,GO:0071038,GO:0071049,GO:0071051'    



In [14]:
prot2go['O81127']

'GO:0005515,GO:0016607,GO:0005681,GO:0008270,GO:0003676,GO:0000398,GO:0000166'

In [15]:
save_dict(prot2go, 'ATprot2go.pkl')

In [16]:
max_golen = 0
for key, value in prot2go.items():
    gos = value.split(',')
    count = 0
    for go in gos:
        if len(go) > 3:
            feature = np.load('../ncbi_allfeatures4go/'+go+'_0.npy')
            count += feature.shape[0]-2
    if max_golen < count:
        max_golen = count
    

In [17]:
max_golen

270